In [1]:
import os

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical   
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import vgg16, inception_v3, resnet50, mobilenet, inception_resnet_v2
import numpy as np
import pandas as pd

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
IMAGE_SHAPE = (160, 160)

In [3]:
def generate_bottlenecks_and_labels(model, image_shape, augment_factor):
    '''
    Self Explanatory
    
    Arguments
    - model: keras.model.Model, usually a pre-existing model excluding top-layers,
             with pre-trained weights
    - image_shape: tuple(x, y)
    - augment_factor: how many times to augment non-test images
    
    Returns a tuple of 2 elements
    - bottlenecks: dict of bottleneck np.ndarrays, by dataset
    - labels: dict of label np.ndarrays, by dataset
    '''
    
    # define augmenations
    transform_parameters = {
        'zx': 0.6,
        'zy': 0.6,
    }
    zoom_gen = ImageDataGenerator()
    zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)
    
    aug_gens = dict()
    aug_gens['train'] = ImageDataGenerator(
            rotation_range=40,
            fill_mode='nearest',
            preprocessing_function=zoom)
    aug_gens['validation'] = ImageDataGenerator(
            rotation_range=40,
            fill_mode='nearest',
            preprocessing_function=zoom)
    aug_gens['test'] = ImageDataGenerator(
            preprocessing_function=zoom)
    
    # get generator per dataset
    ordered_gens = dict()
    kwargs = dict(
        target_size=image_shape,
        batch_size=1,
        class_mode=None,
        shuffle=False
    )
    for key, aug_gen in aug_gens.items():
        ordered_gens[key] = aug_gen.flow_from_directory(
            '../data/data/{}'.format(key), **kwargs)
    
    # generate bottleneck labels after augmentation
    labels = dict()
    for key, gen in ordered_gens.items():
        if key == 'test':
            labels[key] = gen.classes
        else:
            labels[key] = np.tile(gen.classes, augment_factor)

    # generate bottlenecks by dataset
    kwargs = dict(
        verbose=1,
        workers=8,
        use_multiprocessing=True,
    )

    bottlenecks = dict()
    for key, gen in ordered_gens.items():
        print('Preparing {} bottlenecks'.format(key))
        bottlenecks[key] = model.predict_generator(
            gen, steps=len(labels[key]), **kwargs
        )

    return bottlenecks, labels

In [4]:
def get_input_shape(image_shape):
    '''
    Get input shape of conv-nets based on keras backend settings
    
    Returns
    tuple(n1, n2, n3)
    '''
    
    if keras.backend.image_data_format() == 'channels_first':
        return (3,) + image_shape 
    else:
        return image_shape + (3,)

In [5]:
pre_model = mobilenet.MobileNet(
    weights='imagenet',
    include_top=False,
    input_shape=get_input_shape(IMAGE_SHAPE)
)

In [6]:
for layer in pre_model.layers[:-3]:
    layer.trainable = False

In [7]:
bottlenecks, labels = generate_bottlenecks_and_labels(
    pre_model, IMAGE_SHAPE, augment_factor=10)

Found 1682 images belonging to 3 classes.
Found 472 images belonging to 3 classes.
Found 519 images belonging to 3 classes.
Preparing train bottlenecks
16820/16820 [==============================] - 1072s 64ms/step
Preparing validation bottlenecks
4720/4720 [==============================] - 301s 64ms/step
Preparing test bottlenecks
519/519 [==============================] - 32s 62ms/step


In [8]:
'''
np.save(open('train_bottlenecks.npy', 'wb'), train_bottlenecks)
np.save(open('validation_bottlenecks.npy', 'wb'), validation_bottlenecks)
np.save(open('test_bottlenecks.npy', 'wb'), test_bottlenecks)
'''

"\nnp.save(open('train_bottlenecks.npy', 'wb'), train_bottlenecks)\nnp.save(open('validation_bottlenecks.npy', 'wb'), validation_bottlenecks)\nnp.save(open('test_bottlenecks.npy', 'wb'), test_bottlenecks)\n"

In [10]:
top_model = Sequential()
top_model.add(Flatten(input_shape=pre_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))
#top_model.load_weights('bottleneck_fc_model.h5')
top_model.compile(loss='categorical_crossentropy',
                  #optimizer=optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
                  #optimizer=optimizers.SGD(lr=0.01, momentum=0.9),
                  optimizer='rmsprop',
                  metrics=['accuracy'])

In [11]:
# one-hot labels
hot_labels = dict()
for key, label_array in labels.items():
    hot_labels[key] = to_categorical(label_array, num_classes=3)

In [12]:
# train model
batch_size = 16
top_model.fit(bottlenecks['train'], hot_labels['train'],
              validation_data=(bottlenecks['validation'], hot_labels['validation']),
              epochs=25,
              batch_size=batch_size,
              shuffle=True)

Train on 16820 samples, validate on 4720 samples
Epoch 1/25
16820/16820 [==============================] - 59s 4ms/step - loss: 2.3466 - acc: 0.8313 - val_loss: 2.9952 - val_acc: 0.7860
Epoch 2/25
16820/16820 [==============================] - 53s 3ms/step - loss: 0.7987 - acc: 0.9388 - val_loss: 2.6015 - val_acc: 0.8220
Epoch 3/25
16820/16820 [==============================] - 52s 3ms/step - loss: 0.4404 - acc: 0.9647 - val_loss: 2.3027 - val_acc: 0.8475
Epoch 4/25
16820/16820 [==============================] - 51s 3ms/step - loss: 0.3128 - acc: 0.9752 - val_loss: 2.2425 - val_acc: 0.8432
Epoch 5/25
16820/16820 [==============================] - 52s 3ms/step - loss: 0.2149 - acc: 0.9822 - val_loss: 2.3777 - val_acc: 0.8411
Epoch 6/25
16820/16820 [==============================] - 65s 4ms/step - loss: 0.1633 - acc: 0.9866 - val_loss: 2.3282 - val_acc: 0.8453
Epoch 7/25
16820/16820 [==============================] - 1080s 64ms/step - loss: 0.1074 - acc: 0.9912 - val_loss: 2.0914 - val_a

In [13]:
# evaluate model
top_model.evaluate(bottlenecks['test'], hot_labels['test'])

519/519 [==============================] - 0s 558us/step


[2.579845699271715, 0.8285163776493256]

In [14]:
# save weights for model
top_model.save_weights('bottleneck_fc_model.h5')

In [15]:
for layer in pre_model.layers[:-3]:
    layer.trainable = False

# Two Layer

In [54]:
pre_model = mobilenet.MobileNet(
    weights='imagenet',
    include_top=False,
    input_shape=get_input_shape(IMAGE_SHAPE)
)

In [55]:
for layer in pre_model.layers[-3:]:
    layer.trainable = True
for layer in pre_model.layers[:-3]:
    layer.trainable = False

In [56]:
new_model = Sequential()
new_model.add(pre_model)

In [57]:
top_model = Sequential()
top_model.add(Flatten(input_shape=pre_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))

new_model.add(top_model)

In [58]:
top_model.load_weights('bottleneck_fc_model.h5')

In [59]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  #optimizer='rmsprop',
                  metrics=['accuracy'])

In [60]:
batch_size = 16

transform_parameters = {
    'zx': 0.6,
    'zy': 0.6,
}
zoom_gen = ImageDataGenerator()
zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)
gen = ImageDataGenerator(
        preprocessing_function=zoom)
aug_gen = ImageDataGenerator(
        rotation_range=40,
        fill_mode='nearest',
        preprocessing_function=zoom)
aug_gen2 = ImageDataGenerator(
        rotation_range=40,
        fill_mode='nearest',
        preprocessing_function=zoom)

train_image_generator = aug_gen.flow_from_directory(
    '../data/data/train',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size)

validation_image_generator = aug_gen2.flow_from_directory(
    '../data/data/validation',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size)

test_image_generator = gen.flow_from_directory(
    '../data/data/test',
    target_size=(IMAGE_SHAPE),
    batch_size=batch_size,
    shuffle=False)

Found 1682 images belonging to 3 classes.
Found 472 images belonging to 3 classes.
Found 519 images belonging to 3 classes.


In [61]:
current_epoch = 0

In [62]:
epoch_count = 50
new_model.fit_generator(
    train_image_generator,
    epochs=epoch_count,
    validation_data=validation_image_generator,
    workers=8,
    use_multiprocessing=True)
current_epoch += epoch_count

Epoch 1/50
106/106 [==============================] - 93s 875ms/step - loss: 5.7582 - acc: 0.6144 - val_loss: 2.7365 - val_acc: 0.8093
Epoch 2/50
106/106 [==============================] - 82s 776ms/step - loss: 2.2142 - acc: 0.8237 - val_loss: 2.3795 - val_acc: 0.8199
Epoch 3/50
106/106 [==============================] - 79s 745ms/step - loss: 1.3666 - acc: 0.8927 - val_loss: 2.6204 - val_acc: 0.8220
Epoch 4/50
106/106 [==============================] - 79s 744ms/step - loss: 1.2797 - acc: 0.8986 - val_loss: 2.9712 - val_acc: 0.7966
Epoch 5/50
106/106 [==============================] - 79s 747ms/step - loss: 1.1190 - acc: 0.9110 - val_loss: 2.7173 - val_acc: 0.7966
Epoch 6/50
106/106 [==============================] - 80s 751ms/step - loss: 0.8325 - acc: 0.9233 - val_loss: 2.5084 - val_acc: 0.8178
Epoch 7/50
106/106 [==============================] - 80s 755ms/step - loss: 0.8791 - acc: 0.9246 - val_loss: 2.7203 - val_acc: 0.8030
Epoch 8/50
106/106 [==============================] - 8

Process ForkPoolWorker-487:
Process ForkPoolWorker-480:
Process ForkPoolWorker-477:
Process ForkPoolWorker-476:
Process ForkPoolWorker-485:
Process ForkPoolWorker-486:
Process ForkPoolWorker-488:
Process ForkPoolWorker-479:
Process ForkPoolWorker-478:
Process ForkPoolWorker-484:
Process ForkPoolWorker-483:
Process ForkPoolWorker-475:
Process ForkPoolWorker-481:
Process ForkPoolWorker-482:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cella

  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr

  File "/Users/itsnamgyu/Code/cardiac-research/slice_range/venv/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1143, in apply_transform
    fill_mode=self.fill_mode, cval=self.cval)
  File "/Users/itsnamgyu/Code/cardiac-research/slice_range/venv/lib/python3.6/site-packages/keras_preprocessing/image.py", line 335, in apply_affine_transform
    cval=cval) for x_channel in x]
  File "/Users/itsnamgyu/Code/cardiac-research/slice_range/venv/lib/python3.6/site-packages/keras_preprocessing/image.py", line 335, in <listcomp>
    cval=cval) for x_channel in x]
  File "/Users/itsnamgyu/Code/cardiac-research/slice_range/venv/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
KeyboardInterrupt


KeyboardInterrupt: 

In [63]:
new_model.evaluate_generator(test_image_generator)

[2.0761685052805805, 0.838150289017341]

In [ ]:
new_model.predict_generator(train_image_generator)